In [37]:
# downscale-cmip6 project pattern (IOWA - use with staging/impactlab & downscalecmip6 argo cluster)

ERA_OUTPUT_PATTERN = 'gs://clean-b1dbca25/reanalysis/ERA-5/F320/{variable_id}.1995-2015.F320.zarr'

CLEAN_OUTPUT_PATTERN = 'gs://clean-b1dbca25/cmip6/{activity_id}/{institution_id}/{source_id}/{experiment_id}/{member_id}/{table_id}/{variable_id}/{delivery_version}.zarr'

BC_OUTPUT_PATTERN = 'gs://biascorrected-492e989a/stage/{activity_id}/{institution_id}/{source_id}/{experiment_id}/{member_id}/{table_id}/{variable_id}/{delivery_version}.zarr'

DC_OUTPUT_PATTERN = 'gs://downscaled-288ec5ac/outputs/{activity_id}/{institution_id}/{source_id}/{experiment_id}/{member_id}/{table_id}/{variable_id}/{delivery_version}.zarr'


# onyx project pattern (Oregon - use with onyx clusters)
# OUTPUT_PATTERN = (
#     "gs://{CRS_SUPPORT_BUCKET}/internal_datasets/climate/CMIP6/gridded-climate-data/"
#     "R_CIL_GDPCIR-v{delivery_version}/{activity_id}/{institution_id}/{source_id}/"
#     "{experiment_id}/{member_id}/{table_id}/{variable_id}/{delivery_version}.zarr"
# )

ERA_DIAGNOSTICS_PATTERN = 'gs://downscaled-288ec5ac/diagnostics/RELEASE-{delivery_version}/{diagnostics_name}/reanalysis/ERA5/F320/{variable_id}/{delivery_version}.zarr'

DIAGNOSTICS_PATTERN = (
    'gs://downscaled-288ec5ac/diagnostics/RELEASE-{delivery_version}/'
    '{diagnostics_name}/'
    '{activity_id}/{institution_id}/{source_id}/'
    '{experiment_id}/{member_id}/{table_id}/{variable_id}/{delivery_version}.zarr'
)

DEBUG = False

In [5]:
import yaml
with open('/home/jovyan/output/dcmip6_all_paths.yaml', 'r') as f:
    DC6_PATHS = yaml.safe_load(f)

In [8]:
%%capture
try:
    import xclim
except ModuleNotFoundError:
    ! pip install xclim
    import xclim

In [9]:
import os
import dask.distributed as dd
import fsspec
import gcsfs
from geopy.geocoders import Nominatim
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import rhg_compute_tools.kubernetes as rhgk
import rhg_compute_tools.utils as rhgu

from tqdm.auto import tqdm

In [22]:
DELIVERY_VERSION = 'v1.1'

In [10]:
DELIVERY_MODELS = [
    'BCC-CSM2-MR',
    #'FGOALS-g3',
    #'ACCESS-ESM1-5',
    #'ACCESS-CM2',
    #'INM-CM4-8',
    #'INM-CM5-0',
    #'MIROC-ES2L',
    #'MIROC6',
    #'NorESM2-LM',
    #'NorESM2-MM',
    #'GFDL-ESM4',
    #'GFDL-CM4',
    #'NESM3',
]

In [11]:
INSTITUTIONS = {    
    'BCC-CSM2-MR': 'BCC',
    'FGOALS-g3': 'CAS',
    'ACCESS-ESM1-5': 'CSIRO',
    'ACCESS-CM2': 'CSIRO-ARCCSS',
    'INM-CM4-8': 'INM',
    'INM-CM5-0': 'INM',
    'MIROC-ES2L': 'MIROC',
    'MIROC6': 'MIROC',
    'NorESM2-LM': 'NCC',
    'NorESM2-MM': 'NCC',
    'GFDL-ESM4': 'NOAA-GFDL',
    'GFDL-CM4': 'NOAA-GFDL',
    'NESM3': 'NUIST',
}

In [12]:
ENSEMBLE_MEMBERS = {
    'BCC-CSM2-MR': 'r1i1p1f1',
    'FGOALS-g3': 'r1i1p1f1',
    'ACCESS-ESM1-5': 'r1i1p1f1',
    'ACCESS-CM2': 'r1i1p1f1',
    'INM-CM4-8': 'r1i1p1f1',
    'INM-CM5-0': 'r1i1p1f1',
    'MIROC-ES2L': 'r1i1p1f2',
    'MIROC6': 'r1i1p1f1',
    'NorESM2-LM': 'r1i1p1f1',
    'NorESM2-MM': 'r1i1p1f1',
    'GFDL-ESM4': 'r1i1p1f1',
    'GFDL-CM4': 'r1i1p1f1',
    'NESM3': 'r1i1p1f1',
}

In [13]:
GRID_SPECS = {
    "ACCESS-CM2": "gn",
    "MRI-ESM2-0": "gn",
    "CanESM5": "gn",
    "ACCESS-ESM1-5": "gn",
    "MIROC6": "gn",
    "EC-Earth3": "gr",
    "EC-Earth3-Veg-LR": "gr",
    "EC-Earth3-Veg": "gr",
    "MPI-ESM1-2-HR": "gn",
    "CMCC-ESM2": "gn",
    "INM-CM5-0": "gr1",
    "INM-CM4-8": "gr1",
    "MIROC-ES2L": "gn",
    "MPI-ESM1-2-LR": "gn",
    "FGOALS-g3": "gn",
    "BCC-CSM2-MR": "gn",
    "AWI-CM-1-1-MR": "gn",
    "NorESM2-LM": "gn",
    "GFDL-ESM4": "gr1",
    "GFDL-CM4": "gr1",
    "CAMS-CSM1-0": "gn",
    "NorESM2-MM": "gn",
    "NESM3": "gn",
}

In [14]:
HIST_EXTENSION_SCENARIO = {
    "ACCESS-CM2": "ssp370",
    "MRI-ESM2-0": "ssp370",
    "CanESM5": "ssp370",
    "ACCESS-ESM1-5": "ssp370",
    "MIROC6": "ssp370",
    "EC-Earth3": "ssp370",
    "EC-Earth3-Veg-LR": "ssp370",
    "EC-Earth3-Veg": "ssp370",
    "MPI-ESM1-2-HR": "ssp370",
    "CMCC-ESM2": "ssp370",
    "INM-CM5-0": "ssp370",
    "INM-CM4-8": "ssp370",
    "MIROC-ES2L": "ssp370",
    "MPI-ESM1-2-LR": "ssp370",
    "FGOALS-g3": "ssp370",
    "BCC-CSM2-MR": "ssp370",
    "AWI-CM-1-1-MR": "ssp370",
    "NorESM2-LM": "ssp370",
    "GFDL-ESM4": "ssp370",
    "GFDL-CM4": "ssp245",
    "CAMS-CSM1-0": "ssp370",
    "NorESM2-MM": "ssp370",
    "NESM3": "ssp245",
}

In [253]:
client, cluster = rhgk.get_giant_cluster(extra_pip_packages='xclim')

N_WORKERS = 60 # 60
cluster.scale(N_WORKERS)

cluster

In [ ]:
print('https://compute.impactlab.org' + cluster.dashboard_link)

In [254]:
# wait until the workers come online - otherwise we get unbalanced task loading and things start acting all fishy
import time
while len(client.ncores()) < N_WORKERS:
    time.sleep(5)

In [11]:
import dask
dask.config.set(**{'array.slicing.split_large_chunks': False})

In [42]:
def get_fp(model, scenario, variable, diagnostics_type):

    fs = gcsfs.GCSFileSystem(
        token='/opt/gcsfuse_tokens/impactlab-data.json',
        timeout=120,
        cache_timeout=120,
        requests_timeout=120,
        read_timeout=120,
        conn_timeout=120,
    )
    
    if model == 'ERA5':
        fp = ERA_OUTPUT_PATTERN.format(variable_id=variable, 
                                       delivery_version=DELIVERY_VERSION)
        diagnostics_fp = ERA_DIAGNOSTICS_PATTERN.format(variable_id=variable,
                                                        delivery_version=DELIVERY_VERSION,
                                                        diagnostics_name=diagnostics_type)
    else:
        paths = DC6_PATHS[f"{model}-{variable}"][scenario]
        
        if scenario == 'historical':
            activity_id = 'CMIP'
        else:
            activity_id = 'ScenarioMIP'
            
        if 'biascorrected' in diagnostics_type:
            fp = paths['biascorrected']
        elif 'clean' in diagnostics_type:
            fp = paths['clean']
        else: #downscaled delivered
            fp = DC_OUTPUT_PATTERN.format(
                activity_id=activity_id,
                institution_id=INSTITUTIONS[model],
                source_id=model,
                experiment_id=scenario,
                member_id=ENSEMBLE_MEMBERS[model],
                table_id='day',
                variable_id=variable,
                delivery_version=DELIVERY_VERSION,
            )
        

        diagnostics_fp = DIAGNOSTICS_PATTERN.format(
            diagnostics_name=diagnostics_type,
            activity_id=activity_id,
            institution_id=INSTITUTIONS[model],
            source_id=model,
            experiment_id=scenario,
            member_id=ENSEMBLE_MEMBERS[model],
            table_id='day',
            variable_id=variable,
            delivery_version=DELIVERY_VERSION,
        )



    
    return fp, diagnostics_fp, fs

In [17]:
def read_data(model, scenario, variable, diagnostic_type, overwrite, isel_slices=None):
    
    fp, diagnostics_fp, fs = get_fp(model, scenario, variable, diagnostic_type)
    
    if fs.exists(diagnostics_fp) and not overwrite:
        return "output path exists already"
    if not fs.exists(fp):
        return "input path does not exist"
    
    mapper = fs.get_mapper(fp)
    ds = xr.open_zarr(mapper)
    
    if isel_slices is not None:
        ds = ds.isel(isel_slices, drop=False)

    return ds, fs, diagnostics_fp

In [18]:
def get_latlon_to_citynames():
    
    lats = {}
    lons = {}
    geolocator = Nominatim(user_agent="whatever")
    cities = ['Tokyo', 'Delhi', 'Shanghai' , 'Sao Paulo', 'Mexico City',
              'Cairo', 'Dhaka', 'New York', 'Buenos Aires', 'Istanbul',
              'Lagos', 'Paris', 'Moscow', 'Miami', 'Mumbai', 'Manila', 'London']
    for c in cities:
        location = geolocator.geocode(c) 
        lats[c] = location.latitude
        lons[c] = location.longitude
    
    return lats, lons

In [19]:
@rhgu.block_globals(whitelist=[
    'DIAGNOSTICS_PATTERN',
    'OUTPUT_PATTERN',
    'DELIVERY_VERSION',
    'DELIVERY_MODELS',
    'INSTITUTIONS',
    'ENSEMBLE_MEMBERS',
    'GRID_SPECS',
    'HIST_EXTENSION_SCENARIO',
])
def diagnostics(model, scenario, diagnostics_type, variable, overwrite=False, debug=False):
    
    client = dd.get_client()
    
    isel_slices = None
    if debug and 'daily' not in diagnostics_type:
        isel_slices = {'lat': slice(0,2), 'lon': slice(0,2)}
    
    i_o = read_data(model, scenario, variable, diagnostics_type, overwrite, isel_slices)
    
    if isinstance(i_o, str):
        return i_o
    else:
        ds, fs, out = i_o
        
    if 'daily' in diagnostics_type:
        lats, lons = get_latlon_to_citynames()
        summary = ds.sel(lat=xr.DataArray(list(lats.values()), dims='city', coords={"city":list(lats.keys())}), 
                lon=xr.DataArray(list(lons.values()), dims='city', coords={"city":list(lats.keys())}), 
                method='nearest')
        summary.chunk({'time': 365})
    elif 'annual-tasmax-diagnostics' in diagnostics_type :           
        average = ds.groupby('time.year').mean()
        count_hot = ds.where(ds > 308.15).groupby('time.year').count() # > 95F, in Kelvin
        average, count_hot = client.compute(
            [average, count_hot],
            sync=True,
            optimize_graph=True,
            retries=3,
        )
        summary = xr.Dataset({
            'annual_average_tasmax': average['tasmax'],
            'annual_count_above_95F': count_hot['tasmax'],
        })
        summary['annual_average_tasmax'].attrs.update({
            'long_name': 'annual_average_tasmax',
            'units': 'Kelvin',
            'description': 'Annual average maximum daily temperature',
            'cell_method': "ds.groupby('time.year').mean()",
        })
        summary['annual_count_above_95F'].attrs.update({
            'long_name': 'annual_count_above_95F',
            'units': 'day count',
            'description': 'Annual count of days with maximum temperature above 308.15 Kelvin = 95F.',
            'cell_method': "ds.where(ds > 308.15).groupby('time.year').sum()",
        })    
        summary.attrs.update(ds.attrs)
        summary.chunk({'year': 5})  
    elif 'annual-precip-diagnostics' in diagnostics_type:       
        cdd = xclim.indicators.atmos.maximum_consecutive_dry_days(ds['pr'])
        # this is buggy
        #cdd = ds.pr.map_blocks(xclim.indicators.icclim.CDD, template=ds.pr.resample(time='YS').count())
        max_5day_precip = ds.pr.rolling(time=5, center=True).sum().resample(time='YS').max()
        total_seasonal_precip = ds.pr.resample(time='3MS').sum()
        total_annual_precip = total_seasonal_precip.resample(time='YS').sum()
        # this is buggy 
        # cdd, max_5day_precip, total_seasonal_precip, total_annual_precip = client.compute(
        #     [cdd, max_5day_precip, total_seasonal_precip, total_annual_precip],
        #     sync=True,
        #     optimize_graph=True,
        #     retries=3,
        # )
        cdd = cdd.compute()
        max_5day_precip = max_5day_precip.compute()
        total_seasonal_precip = total_seasonal_precip.compute()
        total_annual_precip = total_annual_precip.compute()
        cdd = cdd.assign_coords(time=cdd.time.dt.year).rename({'time': 'year'})
        max_5day_precip = max_5day_precip.assign_coords(time=max_5day_precip.time.dt.year).rename({'time': 'year'})
        total_annual_precip = total_annual_precip.assign_coords(time=total_annual_precip.time.dt.year).rename({'time': 'year'})
        summary = xr.Dataset({
            'cdd': cdd,
            'max_5day_precip': max_5day_precip,
            'total_seasonal_precip': total_seasonal_precip,
            'total_annual_precip': total_annual_precip,
        })
        summary['cdd'].attrs.update({
            'long_name': 'Maximum annual dry day count',
            'units': 'day count',
            'description': 'Maximum annual count of continuous dry days',
            'cell_method': "xclim.indicators.icclim.CDD",
        })
        summary['max_5day_precip'].attrs.update({
            'long_name': 'max_5day_precip',
            'units': 'mm/day',
            'description': 'Maximum annual 5-day cumulative surface precipitation',
            'cell_method': "ds.pr.rolling(time=5, center=True).sum().resample(time='YS').max()",
        })
        summary['total_seasonal_precip'].attrs.update({
            'long_name': 'total_seasonal_precip',
            'units': 'mm/day',
            'description': 'Cumulative seasonal surface precipitation',
            'cell_method': "ds.pr.resample(time='3MS').sum()",
        })
        summary['total_annual_precip'].attrs.update({
            'long_name': 'total_annual_precip',
            'units': 'mm/day',
            'description': 'Cumulative annual surface precipitation',
            'cell_method': "total_seasonal_precip.resample(time='YS').sum()",
        })
        summary.attrs.update(ds.attrs)
        summary.chunk({'year': 5, 'time': 20})
    elif 'annual-tasmin-diagnostics' in diagnostics_type:       
        average = ds.groupby('time.year').mean()
        average = client.compute(
            average,
            sync=True,
            optimize_graph=True,
            retries=3,
        )
        summary = xr.Dataset({
            'annual_average_tasmin': average['tasmin']
        })
        summary['annual_average_tasmin'].attrs.update({
            'long_name': 'annual_average_tasmin',
            'units': 'Kelvin',
            'description': 'Annual average minimum daily temperature',
            'cell_method': "ds.groupby('time.year').mean()",
        })
        summary.attrs.update(ds.attrs)
        summary.chunk({'year': 5})
    else:
        raise ValueError('unknown diagnostic type')
        
    if debug:
        return summary
    else: 
        print(out)
        summary.to_zarr(fs.get_mapper(out), consolidated=True, mode='w')

In [260]:
%%time
try:
    with tqdm(DELIVERY_MODELS) as pbar:
        for m in pbar:
            for s in ['historical']:
                pbar.set_postfix({'model': m, 'scenario': s})
                diagnostics(model=m, scenario=s, diagnostics_type='clean-annual-tasmax-diagnostics', variable='tasmax', debug=DEBUG, overwrite=True)
                diagnostics(model=m, scenario=s, diagnostics_type='biascorrected-annual-tasmax-diagnostics', variable='tasmax', debug=DEBUG, overwrite=True)
                # diagnostics(model=m, scenario=s, diagnostics_type='annual-tasmax-diagnostics', variable='tasmax', debug=DEBUG, overwrite=True)
                # diagnostics(model=m, scenario=s, diagnostics_type='annual-tasmin-diagnostics', variable='tasmin', debug=DEBUG, overwrite=True)
                # diagnostics(model=m, scenario=s, diagnostics_type='daily-tasmax-diagnostics', variable='tasmax', debug=DEBUG, overwrite=True)
                # diagnostics(model=m, scenario=s, diagnostics_type='daily-tasmin-diagnostics', variable='tasmin', debug=DEBUG, overwrite=True)
                # diagnostics(model=m, scenario=s, diagnostics_type='daily-precip-diagnostics', variable='pr', debug=DEBUG, overwrite=True)
    diagnostics(model='ERA5', scenario='historical', diagnostics_type='annual-tasmax-diagnostics', variable='tasmax', debug=DEBUG, overwrite=True)

except Exception:
    client.close()
    cluster.close()
    raise

  0%|          | 0/1 [00:00<?, ?it/s]

CPU times: user 53.7 s, sys: 10.8 s, total: 1min 4s
Wall time: 19min 14s


In [261]:
client.close()
cluster.close()